In [2]:
import torch.nn as nn
from attention import *
import numpy as np
import pandas as pd

In [11]:
import os
os.getcwd()

'c:\\Users\\theju\\transformer-scratch'

In [6]:
!pip install datasets

In [3]:
from torch.utils.data import DataLoader
from datasets import load_from_disk, load_dataset
from tqdm import tqdm

In [4]:
from collections import Counter
import json

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cpu'

In [6]:
np.__version__

'1.26.4'

In [7]:
!pip install pybind11>=2.12


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# !pip install "numpy<2" --force-reinstall

   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   --------------------------- ------------ 11.0/15.8 MB 57.2 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 55.3 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2


  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
ds = load_dataset("roneneldan/TinyStories", trust_remote_code=True)

In [10]:
ds = load_dataset("roneneldan/TinyStories", trust_remote_code=True, download_mode="force_redownload")


FileNotFoundError: Couldn't find a dataset script at /mnt/c/Users/theju/transformer-scratch/roneneldan/TinyStories/TinyStories.py or any data file in the same directory. Couldn't find 'roneneldan/TinyStories' on the Hugging Face Hub either: FileNotFoundError: Dataset 'roneneldan/TinyStories' doesn't exist on the Hub. If the repo is private or gated, make sure to log in with `huggingface-cli login`.

In [11]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 8.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.8/503.8 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 17.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.1.0
    Uninstalling pyarrow-16.1.0:
      Successfully uninstalled pyarrow-16.1.0
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      

In [14]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

FileNotFoundError: Couldn't find a dataset script at /mnt/c/Users/theju/transformer-scratch/roneneldan/TinyStories/TinyStories.py or any data file in the same directory. Couldn't find 'roneneldan/TinyStories' on the Hugging Face Hub either: FileNotFoundError: Dataset 'roneneldan/TinyStories' doesn't exist on the Hub. If the repo is private or gated, make sure to log in with `huggingface-cli login`.

In [9]:
ds.save_to_disk('data')

Saving the dataset (0/4 shards):   0%|          | 0/2119719 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/21990 [00:00<?, ? examples/s]

In [ ]:
ds.sa

In [10]:
ds = load_from_disk('data')
ds.set_format(type="torch")

# Tokenization

In [ ]:
class BytePairEncodingTokenizer:
  def __init__(self,word_split = 'Ġ', padding_token = '<PAD>'):
    self.vocab={}
    self.inverse_vocab={}
    self.tokens=[]
    self.bpe_pairs={}
    self.word_split = word_split
    self.padding_token = padding_token

  def set_vocabulary(self,word_split,special_tokens):
    # Step 1: Setting the unique characters
    unique_characters=[chr(i) for i in range(256)]

    # Step 2: Word spli
    # t characters
    if word_split not in unique_characters:
      unique_characters.append(self.word_split)

    # Step 3: Special tokens
    if special_tokens:
      unique_characters.extend(self.special_tokens)

    return unique_characters

  def get_max_freq_pair(self,tokens):
    pair_counts={}
    pairs=[]
    # Step 1: Gettkng all the token pairs-> (token[i],token[i+1])
    for index in range(len(tokens)-1):
      pairs.append((tokens[index],tokens[index+1]))

    # Step 2: Getting the count of occurences of each of token pairs
    pairs_counts=Counter(pairs)

    # Step 3: Get the token pair whose count is the highest
    max_pair=max(pairs_counts.items(),key=lambda x: x[1])[0]
    return max_pair

  def merge_tokens(self,tokens,max_pair,new_pair_id):
    # In the tokens, check the presence of occurence of max_pair and if exists then replace max_pair with new_pair_id
    # Eg: tokens=[87,76,44,25,38,44,25,19], max_pair=[44,25],  new_pair_id=123
    # Output: [87,76,123,38,123,19]
    new_tokens=[]
    i=0
    while i<=len(tokens)-1:
      if i==len(tokens)-1:
        new_tokens.append(tokens[i])
        break

      elif (tokens[i],tokens[i+1])==max_pair:
        new_tokens.append(new_pair_id)
        i+=2

      else:
        new_tokens.append(tokens[i])
        i+=1
    return new_tokens

  def train(self,text,vocab_size,special_tokens):
    if vocab_size<=258:
      raise ValueError('Please enter a vocab size greater than 258 since this defines the basic set of characters')
    self.special_tokens = special_tokens

    # Setting the vocabulary
    vocab = self.set_vocabulary(self.word_split,self.special_tokens)
    for index,character in enumerate(vocab):
      self.vocab[index]=character
      self.inverse_vocab[character]=index

    # Transforming the text
    ## Step 1: Replacing all thw white-space character
    processed_text=[]
    for index,char in enumerate(text):
      if index!=0 and char==' ':
        processed_text.append(self.word_split)
      if char!=' ':
        processed_text.append(char)
    processed_text="".join(processed_text)

    ## Step 2: Getting the numerical form of token
    self.tokens=[]
    for char in processed_text:
      self.tokens.append(self.inverse_vocab[char])

    ## Step 3: BPE-algorithm
    vocab_length=len(self.vocab)
    for i in range(vocab_length,vocab_size):
      max_pair=self.get_max_freq_pair(self.tokens)
      if max_pair is None:
        break
      self.bpe_pairs[max_pair]=i
      self.tokens=self.merge_tokens(self.tokens,max_pair,i)

    ## Step 4: Update vocab with BPE
    for pair,new_index in self.bpe_pairs.items():
      merged_token=self.vocab[pair[0]]+self.vocab[pair[1]]
      self.vocab[new_index]=merged_token
      self.inverse_vocab[merged_token]=new_index

  def encode(self,text):
    # Step 1: Basically tokens are split into words. Replace all the occurences of "\n" to " <NEWLINE> ". This is to avoid splitting issues.
    tokens_split=text.replace('\n',' <NEWLINE> ').split()
    tokens=[]
    for i in tokens_split:
      if i=='<NEWLINE>':
        tokens.append('\n')
      else:
        tokens.append(i)

    # Step 2: Cleaning of tokens
    ## Eg: 'This is a ball' will be tokenized as ['The','Ġis','Ġa', 'Ġball']
    # Ensures that all the tokens in a line other than the first one will be prefixed with "Ġ" to show the word boundaries
    tokens_cleaned=[]
    for index,token in enumerate(tokens):
      if index>0 and not token.startswith('\n'):
        tokens_cleaned.append(self.word_split+token)
      else:
        tokens_cleaned.append(token)

    # Step 3: Getting the corresponding token IDs from the cleaned tokens
    ## Checks whether tokens exist in the vocabulary. If not, then perform BPE tokenization of the token
    token_ids=[]
    for token in tokens_cleaned:
      if token in self.inverse_vocab.keys():
        token_ids.append(self.inverse_vocab[token])
      else:
        token_ids.extend(self.tokenize_using_bpe(token))
    return token_ids

  def tokenize_using_bpe(self,token):
    # Step 1: Mapping the tokens to their IDs from the vocabulary
    token_ids=[]
    for char in token:
      if char in self.inverse_vocab.keys():
        token_ids.append(self.inverse_vocab[char])
      else:
        token_ids.append(None)

    # Step 2: Check whether token does not exist in Vocabulary- In that case stop
    if None in token_ids:
      token_dict=dict(zip(token_ids,token))
      missing_characters=[]
      for id,ch in token_dict.items():
        if id is None:
          missing_characters.append(ch)
      raise ValueError(f"No token IDs found for the characters:{missing_characters}")

    # Step 3: Now merging
    can_merge=True
    while can_merge and len(token_ids)>1:
      can_merge=False
      i=0
      new_tokens=[]
      """
      Check whether the token pair is part of bpe_pairs occured during training,
      If yes, index = index + 2, else index = index + 1.
      This iteration occurs until there exists no merging exists for all the tokens in token_ids.
      No merging exists means that there are no more possible keys to merge in bpe_pairs.
      """
      while i<len(token_ids)-1:
        pair=(token_ids[i],token_ids[i+1])
        if pair in self.bpe_pairs.keys():
          pair_id=self.bpe_pairs[pair]
          new_tokens.append(pair_id)
          i+=2
          can_merge=True
        else:
          new_tokens.append(token_ids[i])
          i+=1
      if i<len(token_ids):
        new_tokens.append(token_ids[i])
      token_ids=new_tokens

    return token_ids

  def decode(self,token_ids):
    # Step 1: Check whether there are non-existing token IDs
    non_existing_ids=[]
    for id in token_ids:
      if id not in self.vocab.keys():
        non_existing_ids.append(id)
    if len(non_existing_ids)>0:
      raise ValueError(f"No token found for the token IDs:{non_existing_ids}")

    # Step 2: Decoding- Check whether text corresponding to token ID starts with word_split-symbol('Ġ'). If yes replace word_split-symbol with " " else just append the text to string
    final=""
    for id in token_ids:
      text=self.vocab[id]
      if text.startswith(self.word_split):
        final+=" "+text[1:]
      else:
        final+=""+text

    return final

  def save_bpe_vocab_and_merges(self,vocab_path,bpe_path):
    with open(vocab_path,'w',encoding='utf-8') as f:
      json.dump(self.vocab,f,ensure_ascii=False, indent=2)
    with open(bpe_path,'w',encoding='utf-8') as f:
      json.dump([{'pair':list(pair),'id':id } for pair,id in self.bpe_pairs.items()],f,
                ensure_ascii=False, indent=2)

  def load_bpe_vocab_and_merges(self,vocab_path,bpe_path):
    with open(vocab_path,'r',encoding='utf-8') as f:
      loaded_vocab=json.load(f)
      self.vocab = {int(id):token for id,token in loaded_vocab.items()}
      self.inverse_vocab={token:int(id) for id,token in self.vocab.items()}
    with open(bpe_path,'r',encoding='utf-8') as f:
      bpe=json.load(f)
      for merge in bpe:
        self.bpe_pairs[tuple(merge['pair'])]=merge['id']


In [3]:
df = pd.read_csv('spam.csv', encoding='latin-1')
df = df.iloc[:,:2]
df = df.rename(columns = {'v1':'class','v2':'text'})
df.head(2)

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [4]:
n_rows, n_cols = df.shape

In [5]:
train_len = int(n_rows*0.75)
test_len = n_rows-train_len

In [12]:
train_df = df.iloc[:train_len,:]
test_df = df.iloc[train_len:,:]

In [ ]:
train_text = '\n'.join(train_df['text'].to_list())

In [10]:
tokenizer1 = BytePairEncodingTokenizer()
tokenizer1.train(train_text, vocab_size=500, special_tokens={"<|endoftext|>","œ"})

In [16]:
max([len(tokenizer1.encode(i)) for i in train_df['text']])

587

In [ ]:
print(len(tokenizer1.bpe_pairs))

241


## Encoder

In [ ]:
query_shape = (64, 512, 512)  # Example query tensor
key_shape = (3, 512, 512)    # Example key tensor 
value_shape = (3, 512, 512)  # Example value tensor
head_count = 8
model_size = 512
ffn_hidden_dimension = 4096
dropout_rate = 0.1

In [ ]:
class TransformerEmbedding(nn.Module):
    def __init__(self, vocab_size, n_dim):
        super().__init__()
        self.n_dim = n_dim
        self.weights = nn.Parameter(torch.zeros((vocab_size, n_dim)))
        nn.init.uniform_(self.weights)

    def forward(self, x):
        return np.sqrt(self.n_dim) * self.weights[x]

class PositionalEncoding(nn.Module):
    """
    Standard Sinusoidal Positional Encoding.
    
    wavelength: factor to determine the wavelength in the sinusoidal function.
    """
    def __init__(self, wavelength):
        super(PositionalEncoding, self).__init__()
        self.wavelength = wavelength

    def forward(self, x):
        """Given a (... x seq_len x embedding_dim) tensor, returns a (seq_len x embedding_dim) tensor."""
        seq_len, embedding_dim = x.shape[-2], x.shape[-1]
        pe = torch.zeros((seq_len, embedding_dim))
        position = torch.arange(seq_len).unsqueeze(1)
        factor = torch.exp(-math.log(self.wavelength) * torch.arange(0, embedding_dim, 2) / embedding_dim)
        pe[:, 0::2] = torch.sin(position * factor)
        pe[:, 1::2] = torch.cos(position * factor)
        return pe

class ModelEmbeddings(nn.Module):
    def __init__(self,vocab_size, n_dim, wavelength=10000.,dropout_rate=0.1):
        self.pe = PositionalEncoding(wavelength)
        self.te = TransformerEmbedding(vocab_size, n_dim)
        self.dropout = nn.DropOut(dropout_rate)

    def forward(self,x):
        return self.dropout(self.pe(x)+self.te(x))

In [3]:
class Encoder(nn.Module):
    def __init__(self,query_shape,key_shape,value_shape,head_count,model_size,ffn_hidden_dimension,dropout_rate):
        super().__init__()
        self.query_shape = query_shape
        self.key_shape = key_shape
        self.value_shape = value_shape
        self.head_count = head_count
        self.model_size = model_size
        self.ffn_hidden_dimension = ffn_hidden_dimension
        self.multi_head = MultiHeadAttention(self.query_shape,
                                             self.key_shape,
                                             self.value_shape,
                                             self.head_count,
                                             self.model_size)
        self.normal_layer_1 = nn.LayerNorm(self.model_size)
        self.normal_layer_2 = nn.LayerNorm(self.model_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.ffn = FeedForwardNetwork(self.model_size,self.model_size,self.ffn_hidden_dimension)
    
    def forward(self,x):
        x_input = x

        multi_head_output = self.multi_head(x_input,x_input,x_input)
        layer_norm_1_output = self.normal_layer_1(x_input + self.dropout(multi_head_output))

        ffn_output = self.ffn(layer_norm_1_output)
        encoder_output = self.normal_layer_2(layer_norm_1_output + self.dropout(ffn_output))
        
        return encoder_output

In [4]:
encoder = Encoder(
                  query_shape,
                  key_shape,
                  value_shape,
                  head_count,
                  model_size,
                  ffn_hidden_dimension,
                  dropout_rate
                  )

In [5]:
x = torch.randn(3, 64, 512) 
encoder_output = encoder(x)

In [6]:
encoder_output.shape

torch.Size([3, 64, 512])

In [7]:
query_shape = (3, 64, 512)  # Example query tensor
key_shape = (3, 64, 512)    # Example key tensor 
value_shape = (3, 64, 512)  # Example value tensor
head_count = 8
model_size = 512
ffn_hidden_dimension = 4096

## Decoder

In [8]:
class Decoder(nn.Module):
    def __init__(self,query_shape,key_shape,value_shape,head_count,model_size,ffn_hidden_dimension,dropout_rate):
        super().__init__()
        self.query_shape = query_shape
        self.key_shape = key_shape
        self.value_shape = value_shape
        self.head_count = head_count
        self.model_size = model_size
        self.ffn_hidden_dimension = ffn_hidden_dimension
        self.multi_head = MultiHeadAttention(self.query_shape,
                                             self.key_shape,
                                             self.value_shape,
                                             self.head_count,
                                             self.model_size)
        self.masked_multi_head = MultiHeadAttention(self.query_shape,
                                             self.key_shape,
                                             self.value_shape,
                                             self.head_count,
                                             self.model_size)
        self.normal_layer_1 = nn.LayerNorm(self.model_size)
        self.normal_layer_2 = nn.LayerNorm(self.model_size)
        self.normal_layer_3 = nn.LayerNorm(self.model_size)
        self.ffn = FeedForwardNetwork(self.model_size,self.model_size,
                                         self.ffn_hidden_dimension)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self,x, encoder_output):

        """
        1. Masked Multi-Head Attention: 
        This is mainly used to prevent overlooking for futuristic values or predictions of the output. 
        """
        x_input = x
        decoder_ip_sentence_length = x_input.shape[-2]
        mask = torch.triu(torch.ones((decoder_ip_sentence_length,
                                      decoder_ip_sentence_length)),diagonal=1)

        masked_attention = self.masked_multi_head(x_input,x_input,x_input,attention_mask = mask)
        layer_norm_1_output = self.normal_layer_1(x_input + self.dropout(masked_attention))

        """
        2. Multi-Head Attention: 
        This is mainly used to prevent overlooking for futuristic values or predictions of the output. 
        Query-> Decoder based 
        Key-> From the output of encoder
        Value-> From the output of encoder
        """
        multi_head_attention = self.multi_head(layer_norm_1_output,encoder_output,encoder_output)
        layer_norm_2_output = self.normal_layer_2(layer_norm_1_output + self.dropout(multi_head_attention))

        # 3. Feed-forward Network
        decoder_output = self.normal_layer_3(layer_norm_2_output + self.ffn(layer_norm_2_output))
        return decoder_output


In [9]:
decoder = Decoder(
                  query_shape,
                  key_shape,
                  value_shape,
                  head_count,
                  model_size,
                  ffn_hidden_dimension,
                  dropout_rate
                  )

In [17]:
class EncoderDecoderTransformer(nn.Module):
    def __init__(self, num_layers, source_vocab_size, target_vocab_size, query_shape, key_shape, value_shape, head_count, model_size, ffn_hidden_dimension, dropout_rate):
        super().__init__()
        self.encoder_layers = [Encoder(query_shape, 
                                key_shape, 
                                value_shape, 
                                head_count, 
                                model_size, 
                                ffn_hidden_dimension, 
                                dropout_rate) for _ in range(num_layers)]
        self.decoder_layers = [Decoder(query_shape, 
                               key_shape, 
                               value_shape, 
                               head_count, 
                               model_size, 
                               ffn_hidden_dimension, 
                               dropout_rate) for _ in range(num_layers)]
        self.embeddings_ip = ModelEmbeddings(source_vocab_size,model_size)
        self.embeddings_op = ModelEmbeddings(target_vocab_size,model_size)

        self.ffn = nn.Linear(model_size,target_vocab_size)
    
    def forward(self,input_text,output_text):
        
        ip_embedding = self.embeddings_ip(input_text)
        op_embedding = self.embeddings_ip(output_text)

        encoder_output = ip_embedding
        for encoder in self.encoder_layers:
            encoder_output = encoder(ip_embedding)
        
        decoder_output = op_embedding
        for decoder in self.decoder_layers:
            decoder_output = decoder(op_embedding, encoder_output)

        output = self.ffn(decoder_output)

        return output